### Engineering

In [ ]:
#include <iostream>

In [ ]:
int main() {
    int numbers[] = {1, 2, 3, 4};
    
    std::cout << numbers[0];
    
    return 0;
}

In [ ]:
int main() {
    int tensor[3][2] = {
        {1, 2},
        {3, 4},
        {5, 6}
    }
    
    return 0;
}

In [ ]:
thread
process
vector
stream processing

In [1]:
n_microbatches = 4

In [2]:
n_partritions = 3

In [3]:
n_clock_cycles = n_microbatches+n_partritions-1

In [5]:
for clock_idx in range(n_clock_cycles):
    start_partrition = max(clock_idx+1-n_microbatches, 0)
    end_partrition = min(clock_idx+1, n_partritions)
    
    for partrition_idx in range(start_partrition, end_partrition):
        microbatch_idx = clock_idx - partrition_idx 

In [ ]:
step 1: split a mini-batch into micro-batches
step 2: create a new cuda stream for each micro-batches in a device
step 3: pipeline
step 4: comebne

In [ ]:
service, configmap, deployment

In [ ]:
min(clock_idx+1, n_partritions)

In [ ]:
step 1: backward(4, 3), recompute(3, 3)
step 2: backward(3, 3), recompute(2, 3)
step 3: backward(2, 3), recompute(1, 3)

In [ ]:
stream processing
thread-based
process-based
vector

In [6]:
world_size = 16

In [7]:
tensor_model_parallel_size = 2

In [8]:
pipeline_model_parallel_size = 4

In [9]:
num_pipeline_model_paralell_groups = world_size // pipeline_model_parallel_size

In [10]:
num_pipeline_model_paralell_groups

4

In [11]:
data_parallel_groups = []

In [12]:
for i in range(pipeline_model_parallel_size):
    start_rank = i*num_pipeline_model_paralell_groups
    end_rank = (i+1)*num_pipeline_model_paralell_groups
    
    for j in range(tensor_model_parallel_size):
        ranks = list(range(
            start_rank+j,
            end_rank,
            tensor_model_parallel_size
        ))
        
        print(ranks)

[0, 2]
[1, 3]
[4, 6]
[5, 7]
[8, 10]
[9, 11]
[12, 14]
[13, 15]


In [ ]:
broadcast, reduce, scatter, gather

In [ ]:
step 1: load full training data to disk
step 2: specify a list of items
step 3: size
step 4: allocate
step 5: load

In [ ]:
F_{i, j} completed before F_{i+1, j}
B_{i, j} completed before B_{i-1, j}

In [ ]:
max(clock_idx+1-n_microbatches, 0)

In [ ]:
container runtime, kubelet, kube-proxy

In [ ]:
broadcast, scatter, reduce, gather

In [ ]:
loading training data into memory
idxs
size(idxs)
allocate
load

### MechInterp

In [ ]:
act_head = activations[1, 2, :, :]

In [ ]:
step 1: yes and no prompt
step 2: record all the activations of the two prompts
step 3: extract the hidden states
step 4: normalize the hidden states
step 5: convert to probs using the probe

In [ ]:
cache["attn", 2]

In [ ]:
tokens = model.to_tokens(text)

In [ ]:
_, cache = model.run_with_cache(tokens)

In [13]:
def compute_score(pattern, target_pattern):
    score = pattern * target_pattern
    return score.sum(dim=-1) / pattern.sum(dim=-1)

In [ ]:
data = torch.zeros(n_layers, n_heads)

In [ ]:
for layer_idx in range(n_layers):
    hook_name = f"blocks.{layer_idx}.attn.hook_pattern"
    attention_patterns = cache[hook_name]
    
    for head_idx in range(n_heads):
        head_attn_pattern = attention_patterns[0, head_idx, :, :]
        score = compute_score(head_attn_pattern, target_pattern)
        data[layer_idx, head_idx] = score

In [ ]:
step 1: choose a component
step 2: a clean prompt and a corrupted prompt
step 3: record all activations of the clean prompt and corrupted prompt
step 4: patch the activation of the componnet from the corrupted to the clean
step 5: rerun the clean with patched component
step 6: extract the activations of receivers
step 7: rerun the clean prompt and patch the receiver

In [ ]:
text_embeddings = model.embed(tokens)
positional_embeddings = model.pos_embed(tokens)

In [ ]:
residual = text_embeddings + positional_embeddings

In [ ]:
for block in model.blocks:
    residual = block(residual)

In [ ]:
residual = model.ln_final(residual)

In [ ]:
logits = model.unembed(residual)

In [ ]:
corrupted_prompt = "A told B: 'Persistence is all you need.' C replied back to "

In [ ]:
tokens = model.to_tokens(prompt)

In [ ]:
corrupted_tokens = model.to_tokens(corrupted_prompt)

In [ ]:
_, clean_activations = model.run_with_cache(tokens)
_, corrupted_activations = model.run_with_cache(corrupted_tokens)

In [14]:
from transformer_lens import utils

In [15]:
head_idx, layer_idx = 6, 9

In [16]:
hook_name = utils.get_act_name("attn", layer_idx)

In [ ]:
corrupted_head_activations = corrupted_activations[hook_name][0, head_idx, :, :]

In [ ]:
def patch_head_activations(activations, hook):
    activations[0, head_idx, :, :] = corrupted_head_activations
    return activations

In [ ]:
corrupted_activations = model.run_with_hooks(
    tokens,
    fwd_hooks=[(hook_name, patch_head_activations)]
)

In [ ]:
corrupted_receiver_activations = corrupted_activations[receiver_hook_name]

In [ ]:
def patch_receiver_activations(activations, hook):
    activations = corrupted_receiver_activations
    return activations

In [ ]:
model.add_hook(receiver_hook_name, patch_receiver_activations)

In [ ]:
corrupted_logits = model(tokens)

In [ ]:
clean_logits = model(tokens)

In [ ]:
def compute_logit_difference(clean_logits, corrupted_logits, target_token):
    return clean_logits[:, -1, :][target_token, :] - corrupted_logits[:, -1, :][target_token, :]

In [17]:
from fastapi import status

In [ ]:
@app.get("/items", status_code=status.HTTP_201_CREATED)
def get_items():
    pass

In [18]:
from typing import Optional

In [ ]:
@app.get("/files")
def files(is_deleted: Optional[bool] = False):
    pass

In [19]:
import torch
from torch import nn
import torch.nn.functional as F

In [ ]:
class MultiplyConstant(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        ctx.input = input
        return input
    
    @staticmethod
    def backward(ctx, grad_input):
        return ctx.input + 1

In [20]:
from fastapi import HTTPException

In [ ]:
@app.get("/blog")
def blog():
    raise HTTPException(
        status_code=status.HTTP_404_NOT_FOUND,
        detail="Blog not found"
    )

In [ ]:
@app.get("/movies/{name}")
def get_movies(name):
    return name

In [21]:
n_microbatches

4

In [22]:
n_partritions

3

In [29]:
n_clock_cycles = n_microbatches + n_partritions - 1

In [42]:
n_clock_cycles

6

In [48]:
for clock_idx in range(n_clock_cycles):
    start_partrition = max(clock_idx+1-n_microbatches, 0)
    end_partrition = min(clock_idx+1, n_partritions)
    
    # print(start_partrition, end_partrition)
    
    tasks = []
    for partrition_idx in range(start_partrition, end_partrition):
        microbatch_idx = clock_idx - partrition_idx
        tasks.append((microbatch_idx, partrition_idx))
    
    print(tasks)

[(0, 0)]
[(1, 0), (0, 1)]
[(2, 0), (1, 1), (0, 2)]
[(3, 0), (2, 1), (1, 2)]
[(3, 1), (2, 2)]
[(3, 2)]


In [39]:
tasks

[(5, 0), (4, 1), (3, 2), (2, 3), (1, 4), (0, 5)]

In [ ]:
population, conditioned, motor